In [3]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler

In [15]:
test_df = pd.read_csv('../data/test.csv')

test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [21]:
def preprocess_data(df):
    # for col in ['pclass', 'age', 'sibsp', 'parch']:
    #     col_mean = df.loc[df[col] != -1, col].mean().astype('int')
    #     df.loc[df[col] == -1, col] = col_mean

    # for col in ['embarked', 'name', 'sex', 'ticket', 'cabin']:
    #     df = df[df[col] != "Unknown"]
    df.dropna(inplace=True)
    # Only drop columns that actually exist
    cols_to_drop = ['name', 'ticket', 'passenger_id']
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

    cat_col = df.select_dtypes(include="object").columns.to_list()

    df = pd.get_dummies(df, columns=cat_col, drop_first=True).astype('int')

    return df

In [22]:
test_processed = preprocess_data(test_df)

In [ ]:
model = joblib.load('../models/titanic_rf.joblib')
model

RandomForestClassifier()

In [31]:
def predict_data(model_path, test_data_path):
    try:
        model = joblib.load(model_path)

        test_df = pd.read_csv(test_data_path)
        processed = preprocess_data(test_df)

        processed = processed.reindex(columns=model.feature_names_in_, fill_value=0)
        
        predictions = model.predict(processed)
        
        test_pred = pd.DataFrame({
            'PassengerId': test_df['PassengerId'],
            'Survived': predictions
        })
        
        print("Predictions saved successfully!")
        return test_pred
        
    except Exception as e:
        print(f"Prediction failed: {str(e)}")
        return None

In [32]:
predict_data('../models/titanic_rf.joblib', '../data/test.csv')

Predictions saved successfully!


,PassengerId,Survived
12,904,1
14,906,1
24,916,1
26,918,1
28,920,1
...,...,...
404,1296,1
405,1297,1
407,1299,1
411,1303,1
